

## Predict if someone makes more or less than 50k using the adult dataset

   * Note that this dataset strictly refers to people living in the US






In [1]:
# Importing the libraries
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import tensorflow as tf
import torch
import torch.utils.data as data_utils
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# !pip install torchsummary 
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
torch.backends.cudnn.benchmark=True

/home/anirudh/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Loading the Training and Testing Data using Data Loader 

In [2]:
# Loading the Training and Testing Data
trainImages = np.load('./Adult/data.npy').astype('float32')
trainLabels = np.load('./Adult/labels.npy').astype('float32')

# # Normalizing the data
# mx = np.max(np.float32(trainImages.flatten()))
# Splitting the Data into Training and Test Data    trainImages.max(axis=0)
X_train, X_test,Y_train,Y_test = train_test_split(trainImages,trainLabels, test_size=0.15, shuffle = True)

# batch_size
batch = 256

# Train Data Loader
train = data_utils.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(Y_train))
train_loader = data_utils.DataLoader(train, batch_size=batch, shuffle=True)

# Test Data loader
test = data_utils.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(Y_test))
test_loader = data_utils.DataLoader(test)



In [3]:
X_train.shape

(38438, 67)

In [4]:
X_test.shape

(6784, 67)

# Showing the Training Data after Normalizing

In [5]:
# Preview the training data
for batch_idx, (data, target) in enumerate(train_loader):
    print(batch_idx)
#     print("Training Data")
    print(data[0][0])
#     print("Testing Data")
#     print(target[0])
#     print(data.sha?pe)
#     break
    

0
tensor(22.)
1
tensor(21.)
2
tensor(52.)
3
tensor(32.)
4
tensor(51.)
5
tensor(26.)
6
tensor(21.)
7
tensor(36.)
8
tensor(62.)
9
tensor(51.)
10
tensor(25.)
11
tensor(30.)
12
tensor(26.)
13
tensor(44.)
14
tensor(39.)
15
tensor(17.)
16
tensor(50.)
17
tensor(50.)
18
tensor(17.)
19
tensor(31.)
20
tensor(36.)
21
tensor(46.)
22
tensor(19.)
23
tensor(37.)
24
tensor(27.)
25
tensor(42.)
26
tensor(30.)
27
tensor(25.)
28
tensor(72.)
29
tensor(36.)
30
tensor(45.)
31
tensor(58.)
32
tensor(24.)
33
tensor(26.)
34
tensor(41.)
35
tensor(24.)
36
tensor(37.)
37
tensor(20.)
38
tensor(41.)
39
tensor(66.)
40
tensor(52.)
41
tensor(23.)
42
tensor(35.)
43
tensor(23.)
44
tensor(64.)
45
tensor(38.)
46
tensor(36.)
47
tensor(69.)
48
tensor(34.)
49
tensor(56.)
50
tensor(37.)
51
tensor(30.)
52
tensor(32.)
53
tensor(72.)
54
tensor(32.)
55
tensor(23.)
56
tensor(39.)
57
tensor(37.)
58
tensor(45.)
59
tensor(31.)
60
tensor(38.)
61
tensor(23.)
62
tensor(30.)
63
tensor(23.)
64
tensor(19.)
65
tensor(54.)
66
tensor(43.)
67
te

## Defining Convolution Neural Network

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(67, 128)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 32)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(32, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        inp = self.fc1(input_)
        x = self.relu1(inp)
        x = self.dout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dout2(x)
        x = self.fc3(x)
        x = self.prelu(x)
        x = self.out(x)
        y = (self.out_act(x))
        return y
net = Net()
net.to(device)

Net(
  (fc1): Linear(in_features=67, out_features=128, bias=True)
  (relu1): ReLU()
  (dout): Dropout(p=0.2)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (dout2): Dropout(p=0.2)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (prelu): PReLU(num_parameters=1)
  (out): Linear(in_features=32, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [7]:
summary(net,input_size = (1,67),batch_size=batch)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [256, 1, 128]           8,704
              ReLU-2              [256, 1, 128]               0
           Dropout-3              [256, 1, 128]               0
            Linear-4               [256, 1, 64]           8,256
              ReLU-5               [256, 1, 64]               0
           Dropout-6               [256, 1, 64]               0
            Linear-7               [256, 1, 32]           2,080
             PReLU-8               [256, 1, 32]               1
            Linear-9                [256, 1, 1]              33
          Sigmoid-10                [256, 1, 1]               0
Total params: 19,074
Trainable params: 19,074
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.07
Forward/backward pass size (MB): 1.25
Params size (MB): 0.07
Estimated Tot

In [8]:
params = list(net.parameters())
# print(len(params))
# print(params[0].size())  # conv1's .weight

# params

##  Define a Loss function and optimizer

In [9]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
train_loss =[]
val_loss = []
train_accu = []
test_accu=[]

In [10]:
output = net(data)
#         print(target.dtype,output.dtype)
target = target.view(-1,1)
loss = criterion(output, target)
loss.backward()
optimizer.step()
torch.sum(output==target)


tensor(0)


## Training and Testing

In [13]:
def train(model, train_loader, optimizer, epoch,device):
    model.train()
    criterion = nn.BCELoss()
    training_loss = 0
    train_correct = 0
    lo = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
#         print(target.dtype,output.dtype)
        target = target.view(-1,1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
#         print(loss.item())
        lo.append(loss.item())
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        train_correct += torch.sum(torch.round(output)==target)
#         print(train_correct)
    train_accu.append(100. * train_correct / len(train_loader.dataset))
    train_loss.append(np.mean(lo))

def test(model, test_loader,device):
    model.eval()
    criterion = nn.BCELoss()
    test_loss = 0
    correct = 0
    testlo = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            target = target.view(-1,1)
#             test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            test_loss = criterion(output, target)
            testlo.append(test_loss.item())
#             print(test_loss)
#             print()
            correct += torch.sum(torch.round(output)==target)
#     print(100. * correct / len(test_loader.dataset))
    test_accu.append(100. * correct / len(test_loader.dataset))
    val_loss.append(np.mean(testlo))
#     print(test_loss)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        np.mean(testlo), correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Training the Model

In [14]:
epochs = 50
for epoch in range(1, epochs + 1):
        train(net, train_loader, optimizer, epoch,device)
        test(net, test_loader,device)

Train Epoch: 1 [0/38438 (0%)]	Loss: 0.534055
Train Epoch: 1 [2560/38438 (7%)]	Loss: 0.531182
Train Epoch: 1 [5120/38438 (13%)]	Loss: 0.492474
Train Epoch: 1 [7680/38438 (20%)]	Loss: 0.572266
Train Epoch: 1 [10240/38438 (26%)]	Loss: 0.596717
Train Epoch: 1 [12800/38438 (33%)]	Loss: 0.560076
Train Epoch: 1 [15360/38438 (40%)]	Loss: 0.572222
Train Epoch: 1 [17920/38438 (46%)]	Loss: 0.550599
Train Epoch: 1 [20480/38438 (53%)]	Loss: 0.568900
Train Epoch: 1 [23040/38438 (60%)]	Loss: 0.603931
Train Epoch: 1 [25600/38438 (66%)]	Loss: 0.589733
Train Epoch: 1 [28160/38438 (73%)]	Loss: 0.604763
Train Epoch: 1 [30720/38438 (79%)]	Loss: 0.530749
Train Epoch: 1 [33280/38438 (86%)]	Loss: 0.550177
Train Epoch: 1 [35840/38438 (93%)]	Loss: 0.550723
Train Epoch: 1 [5700/38438 (99%)]	Loss: 0.518731

Test set: Average loss: 0.5488, Accuracy: 5189/6784 (76%)

Train Epoch: 2 [0/38438 (0%)]	Loss: 0.565936
Train Epoch: 2 [2560/38438 (7%)]	Loss: 0.608869
Train Epoch: 2 [5120/38438 (13%)]	Loss: 0.585071
Train Ep

Train Epoch: 10 [33280/38438 (86%)]	Loss: 0.495876
Train Epoch: 10 [35840/38438 (93%)]	Loss: 0.468283
Train Epoch: 10 [5700/38438 (99%)]	Loss: 0.556391

Test set: Average loss: 0.4966, Accuracy: 5263/6784 (77%)

Train Epoch: 11 [0/38438 (0%)]	Loss: 0.487889
Train Epoch: 11 [2560/38438 (7%)]	Loss: 0.511873
Train Epoch: 11 [5120/38438 (13%)]	Loss: 0.501824
Train Epoch: 11 [7680/38438 (20%)]	Loss: 0.492425
Train Epoch: 11 [10240/38438 (26%)]	Loss: 0.527978
Train Epoch: 11 [12800/38438 (33%)]	Loss: 0.529602
Train Epoch: 11 [15360/38438 (40%)]	Loss: 0.499598
Train Epoch: 11 [17920/38438 (46%)]	Loss: 0.526091
Train Epoch: 11 [20480/38438 (53%)]	Loss: 0.484086
Train Epoch: 11 [23040/38438 (60%)]	Loss: 0.520601
Train Epoch: 11 [25600/38438 (66%)]	Loss: 0.514094
Train Epoch: 11 [28160/38438 (73%)]	Loss: 0.508636
Train Epoch: 11 [30720/38438 (79%)]	Loss: 0.538380
Train Epoch: 11 [33280/38438 (86%)]	Loss: 0.538152
Train Epoch: 11 [35840/38438 (93%)]	Loss: 0.518066
Train Epoch: 11 [5700/38438 (99%

Train Epoch: 20 [15360/38438 (40%)]	Loss: 0.443870
Train Epoch: 20 [17920/38438 (46%)]	Loss: 0.410905
Train Epoch: 20 [20480/38438 (53%)]	Loss: 0.490082
Train Epoch: 20 [23040/38438 (60%)]	Loss: 0.444657
Train Epoch: 20 [25600/38438 (66%)]	Loss: 0.376187
Train Epoch: 20 [28160/38438 (73%)]	Loss: 0.422325
Train Epoch: 20 [30720/38438 (79%)]	Loss: 0.431807
Train Epoch: 20 [33280/38438 (86%)]	Loss: 0.434312
Train Epoch: 20 [35840/38438 (93%)]	Loss: 0.430517
Train Epoch: 20 [5700/38438 (99%)]	Loss: 0.429172

Test set: Average loss: 0.3909, Accuracy: 5599/6784 (82%)

Train Epoch: 21 [0/38438 (0%)]	Loss: 0.390511
Train Epoch: 21 [2560/38438 (7%)]	Loss: 0.424740
Train Epoch: 21 [5120/38438 (13%)]	Loss: 0.507446
Train Epoch: 21 [7680/38438 (20%)]	Loss: 0.420381
Train Epoch: 21 [10240/38438 (26%)]	Loss: 0.320159
Train Epoch: 21 [12800/38438 (33%)]	Loss: 0.373794
Train Epoch: 21 [15360/38438 (40%)]	Loss: 0.396464
Train Epoch: 21 [17920/38438 (46%)]	Loss: 0.430644
Train Epoch: 21 [20480/38438 (53

Train Epoch: 29 [5700/38438 (99%)]	Loss: 0.362574

Test set: Average loss: 0.3696, Accuracy: 5653/6784 (83%)

Train Epoch: 30 [0/38438 (0%)]	Loss: 0.404531
Train Epoch: 30 [2560/38438 (7%)]	Loss: 0.378681
Train Epoch: 30 [5120/38438 (13%)]	Loss: 0.425880
Train Epoch: 30 [7680/38438 (20%)]	Loss: 0.422611
Train Epoch: 30 [10240/38438 (26%)]	Loss: 0.371961
Train Epoch: 30 [12800/38438 (33%)]	Loss: 0.428585
Train Epoch: 30 [15360/38438 (40%)]	Loss: 0.374798
Train Epoch: 30 [17920/38438 (46%)]	Loss: 0.433915
Train Epoch: 30 [20480/38438 (53%)]	Loss: 0.362109
Train Epoch: 30 [23040/38438 (60%)]	Loss: 0.417547
Train Epoch: 30 [25600/38438 (66%)]	Loss: 0.410114
Train Epoch: 30 [28160/38438 (73%)]	Loss: 0.412637
Train Epoch: 30 [30720/38438 (79%)]	Loss: 0.382492
Train Epoch: 30 [33280/38438 (86%)]	Loss: 0.369602
Train Epoch: 30 [35840/38438 (93%)]	Loss: 0.403172
Train Epoch: 30 [5700/38438 (99%)]	Loss: 0.452808

Test set: Average loss: 0.3717, Accuracy: 5595/6784 (82%)

Train Epoch: 31 [0/38438

Train Epoch: 39 [20480/38438 (53%)]	Loss: 0.431431
Train Epoch: 39 [23040/38438 (60%)]	Loss: 0.377665
Train Epoch: 39 [25600/38438 (66%)]	Loss: 0.393658
Train Epoch: 39 [28160/38438 (73%)]	Loss: 0.459615
Train Epoch: 39 [30720/38438 (79%)]	Loss: 0.404904
Train Epoch: 39 [33280/38438 (86%)]	Loss: 0.356220
Train Epoch: 39 [35840/38438 (93%)]	Loss: 0.386334
Train Epoch: 39 [5700/38438 (99%)]	Loss: 0.494294

Test set: Average loss: 0.3621, Accuracy: 5691/6784 (83%)

Train Epoch: 40 [0/38438 (0%)]	Loss: 0.362819
Train Epoch: 40 [2560/38438 (7%)]	Loss: 0.392277
Train Epoch: 40 [5120/38438 (13%)]	Loss: 0.313717
Train Epoch: 40 [7680/38438 (20%)]	Loss: 0.358448
Train Epoch: 40 [10240/38438 (26%)]	Loss: 0.431412
Train Epoch: 40 [12800/38438 (33%)]	Loss: 0.347885
Train Epoch: 40 [15360/38438 (40%)]	Loss: 0.412184
Train Epoch: 40 [17920/38438 (46%)]	Loss: 0.385479
Train Epoch: 40 [20480/38438 (53%)]	Loss: 0.416774
Train Epoch: 40 [23040/38438 (60%)]	Loss: 0.393878
Train Epoch: 40 [25600/38438 (66


Test set: Average loss: 0.3497, Accuracy: 5695/6784 (83%)

Train Epoch: 49 [0/38438 (0%)]	Loss: 0.325700
Train Epoch: 49 [2560/38438 (7%)]	Loss: 0.346312
Train Epoch: 49 [5120/38438 (13%)]	Loss: 0.354382
Train Epoch: 49 [7680/38438 (20%)]	Loss: 0.333532
Train Epoch: 49 [10240/38438 (26%)]	Loss: 0.367780
Train Epoch: 49 [12800/38438 (33%)]	Loss: 0.394387
Train Epoch: 49 [15360/38438 (40%)]	Loss: 0.391301
Train Epoch: 49 [17920/38438 (46%)]	Loss: 0.361632
Train Epoch: 49 [20480/38438 (53%)]	Loss: 0.371136
Train Epoch: 49 [23040/38438 (60%)]	Loss: 0.306512
Train Epoch: 49 [25600/38438 (66%)]	Loss: 0.394374
Train Epoch: 49 [28160/38438 (73%)]	Loss: 0.339978
Train Epoch: 49 [30720/38438 (79%)]	Loss: 0.378826
Train Epoch: 49 [33280/38438 (86%)]	Loss: 0.362177
Train Epoch: 49 [35840/38438 (93%)]	Loss: 0.391493
Train Epoch: 49 [5700/38438 (99%)]	Loss: 0.433938

Test set: Average loss: 0.3523, Accuracy: 5696/6784 (83%)

Train Epoch: 50 [0/38438 (0%)]	Loss: 0.368141
Train Epoch: 50 [2560/38438 

In [ ]:
# Evaluate Accuracy
print('Training Loss:', train_loss[-1])
print('Training Accuracy:', train_accu[-1])
print()
print('Test Loss:', val_loss[-1])
print('Testing Accuracy:', test_accu[-1])
print()

plt.plot(train_loss,'r', label='Training Loss')
plt.plot(val_loss,'b', label='Testing Loss')
plt.title('Test Loss' + str(val_loss[-1]))
plt.legend()
plt.show()
plt.plot(train_accu,'r', label='Training accuracy')
plt.plot(test_accu,'b', label='Testing accuracy')
plt.title('Test Accuracy : '+ str(test_accu[-1]))
plt.legend()
plt.show()

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
model_ft

In [ ]:
summary(model_ft,(3,32,32))